In [2]:
%cd /scratch/bruingjde/SNAM2021-code/

import os
import typing

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.pipeline
import sklearn.preprocessing
from tqdm.auto import tqdm

import tlp

feature_name = 'K'

# NOT USED, TOO EXPENSIVE!
# NX implementation could possibly make use of https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.spsolve.html

/scratch/bruingjde/SNAM2021-code


In [28]:
# # Delete
ok = True
for entry in os.scandir('data'):
  filepath = os.path.join(entry, 'features', feature_name + '.pkl')
  print(filepath)
  if os.path.isfile(filepath): os.remove(filepath)

data/25/features/K.pkl
data/23/features/K.pkl
data/07/features/K.pkl
data/04/features/K.pkl
data/10/features/K.pkl
data/12/features/K.pkl
data/24/features/K.pkl
data/08/features/K.pkl
data/22/features/K.pkl
data/15/features/K.pkl
data/30/features/K.pkl
data/21/features/K.pkl
data/11/features/K.pkl
data/28/features/K.pkl
data/13/features/K.pkl
data/29/features/K.pkl
data/01/features/K.pkl
data/17/features/K.pkl
data/18/features/K.pkl
data/20/features/K.pkl
data/03/features/K.pkl
data/26/features/K.pkl
data/05/features/K.pkl
data/19/features/K.pkl
data/14/features/K.pkl
data/06/features/K.pkl
data/27/features/K.pkl
data/02/features/K.pkl
data/09/features/K.pkl
data/16/features/K.pkl


In [29]:
# Check
entries = sorted(os.scandir('data'), key=lambda x: x.name)
ok = True
for entry in entries:
  if not os.path.isfile(os.path.join(entry, 'features', feature_name + '.pkl')):
    ok = False
    print(entry)
if ok: print('Ok!')

<DirEntry '01'>
<DirEntry '02'>
<DirEntry '03'>
<DirEntry '04'>
<DirEntry '05'>
<DirEntry '06'>
<DirEntry '07'>
<DirEntry '08'>
<DirEntry '09'>
<DirEntry '10'>
<DirEntry '11'>
<DirEntry '12'>
<DirEntry '13'>
<DirEntry '14'>
<DirEntry '15'>
<DirEntry '16'>
<DirEntry '17'>
<DirEntry '18'>
<DirEntry '19'>
<DirEntry '20'>
<DirEntry '21'>
<DirEntry '22'>
<DirEntry '23'>
<DirEntry '24'>
<DirEntry '25'>
<DirEntry '26'>
<DirEntry '27'>
<DirEntry '28'>
<DirEntry '29'>
<DirEntry '30'>


In [18]:
def K(path: str, verbose: bool = False, feature_name=feature_name):
  # Check if file exists
  feature_path = os.path.join(path, 'features')
  file = os.path.join(feature_path, f'{feature_name}.pkl')
  if os.path.isfile(file): return 
  
  os.makedirs(feature_path, exist_ok=True)

  # Read in
  edgelist_mature_file = os.path.join(path, 'edgelist_mature.pkl')
  if verbose: print(f'Read {edgelist_mature_file}')
  edgelist_mature = pd.read_pickle(edgelist_mature_file)
  
  instances_file = os.path.join(path, 'instances_sampled.npy')
  if verbose: print(f'Read {instances_file}')  
  instances_sampled = np.load(instances_file)

  # Get all unique nodes in instances
#   nodes = {node for instance in instances_sampled for node in instance}
  
  # Get graph
  G = nx.from_pandas_edgelist(edgelist_mature)
  
  # Calculate per node
  value_per_node = nx.katz_centrality_numpy(G)
  
  value_per_instance = [(value_per_node[u], value_per_node[v]) for u, v in instances_sampled]
  
  def diff(x): return abs(x[1]-x[0]) 
  
  results = {
    tlp.Experiment(feature=feature_name, time_aware=False, nodepair_strategy=strategy_str): (
      np.array([strategy_func(instance) for instance in value_per_instance])) 
    for strategy_str, strategy_func in {'sum': sum, 'diff': diff, 'max': max, 'min': min}.items()
  }
  
  joblib.dump(results, os.path.join(feature_path, f'{feature_name}.pkl'))

In [16]:
for entry in tqdm(entries):
  K(entry, verbose=True)

  0%|          | 0/30 [00:00<?, ?it/s]

Read data/01/edgelist_mature.pkl
Read data/01/instances_sampled.npy


PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 1000 iterations')

In [19]:
entries = sorted(os.scandir('data'), key=lambda x: x.name)
tlp.ProgressParallel(n_jobs=len(entries), total=len(entries))(
  joblib.delayed(K)(entry.path) for entry in entries
)

  0%|          | 0/30 [00:00<?, ?it/s]

MemoryError: Unable to allocate 1.32 TiB for an array with shape (425283, 425283) and data type int64